# Michaelis-Menten with different types of inhibition: Interactive Plots

In [1]:
import numpy as np
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data = pd.read_csv("vmkmki_mmdata.csv")
data = data.rename(columns={"S": "s", "I": "i"})

## Note on the meaning of the data and columns
- **s** represents the concentration of the substrate
- **i** represents the concentration of the inhibitor
- **v** represents the reaction rate

In [3]:
def noncompetitive_inhibition(vmax, s, m_const, i, i_const):
    alpha = 1 + i / i_const
    return ((vmax / alpha) * s) / (s + m_const)


def competitive_inhibition(vmax, s, m_const, i, i_const):
    alpha = 1 + i / i_const
    return (vmax * s) / (alpha * m_const + s)


def uncompetitive_inhibition(vmax, s, m_const, i, i_const):
    alpha = 1 + i / i_const
    return ((vmax / alpha) * s) / ((m_const / alpha) + s)

## Lineweaver Burk Plots

With the data only providing the terms S, V_max, and I and not the constants of K_m and K_i we need to calculate them separately using our experimental data.

Note: K_m can be used to solve for K_i depending on the type of the inhibition

In [4]:
def km_solve(df):
    v = 1 / df["v"]
    v = v.replace({np.inf: 0})
    s = 1 / df["s"]
    s = s.replace({np.inf: 0})

    linmod = sm.OLS(v, s).fit()
    return linmod.params.iloc[0] * df["v"]

In [5]:
m_const = km_solve(data)
i_const = m_const * (data["i"] / (m_const - 1))

In [6]:
data = data.eval(
    """ 
k_m = @m_const
k_i = @i_const
"""
)
obs = data[data["s"] > 0]

In [7]:
ni_v = noncompetitive_inhibition(obs["v"], obs["s"], obs["k_m"], obs["i"], obs["k_i"])
nc_obs = obs.eval("ni_v = @ni_v").dropna()

In [8]:
ci_v = competitive_inhibition(obs["v"], obs["s"], obs["k_m"], obs["i"], obs["k_i"])
c_obs = obs.eval("ci_v = @ci_v").dropna()

In [9]:
ui_v = uncompetitive_inhibition(obs["v"], obs["s"], obs["k_m"], obs["i"], obs["k_i"])
u_obs = obs.eval("ui_v = @ui_v").dropna()

# Interactive Plots

## Noncompetitive Inhibition

In [10]:
app = Dash(__name__)

app.layout = html.Div(
    [
        html.H1(
            "Michaelis-Menten (noncompetitive inhibition) by concentration of inhibitor"
        ),
        html.Label("Select concentration of inhibitor:"),
        dcc.Dropdown(
            id="i-dropdown",
            options=[{"label": val, "value": val} for val in nc_obs["i"].unique()],
            value=nc_obs["i"].unique()[0],
        ),
        dcc.Graph(id="scatter-plot"),
    ]
)


# Callback to update the plot
@app.callback(Output("scatter-plot", "figure"), [Input("i-dropdown", "value")])
def update_plot(selected_i):
    filtered_data = nc_obs[nc_obs["i"] == selected_i]
    fig = px.scatter(
        filtered_data,
        x=np.log(filtered_data["s"]),
        y="ni_v",
        color="s",
        title=f"Scatter plot for inhibitor concentration = {selected_i}",
        labels={
            "ni_v": "Reaction velocity (v)",
            "x": "Log of substrate concentration",
            "s": "Substrate concentration (s)",
        },
        trendline="ols",
        trendline_options=dict(log_x=True),
    )
    return fig


# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)

## Competitive Inhibition

In [11]:
app2 = Dash(__name__)

app2.layout = html.Div(
    [
        html.H1(
            "Michaelis-Menten (competitive inhibition) by concentration of inhibitor"
        ),
        html.Label("Select concentration of inhibitor:"),
        dcc.Dropdown(
            id="i-dropdown",
            options=[{"label": val, "value": val} for val in c_obs["i"].unique()],
            value=c_obs["i"].unique()[0],
        ),
        dcc.Graph(id="scatter-plot"),
    ]
)


# Callback to update the plot
@app2.callback(Output("scatter-plot", "figure"), [Input("i-dropdown", "value")])
def update_plot(selected_i):
    filtered_data = c_obs[c_obs["i"] == selected_i]
    fig = px.scatter(
        filtered_data,
        x=np.log(filtered_data["s"]),
        y="ci_v",
        color="s",
        title=f"Scatter plot for inhibitor concentration = {selected_i}",
        labels={
            "ci_v": "Reaction velocity (v)",
            "x": "Log of substrate concentration",
            "s": "Substrate concentration (s)",
        },
        trendline="ols",
        trendline_options=dict(log_x=True),
    )
    return fig


# Run the app
if __name__ == "__main__":
    app2.run_server(debug=True)

## Uncompetitive inhibition

In [12]:
app3 = Dash(__name__)

app3.layout = html.Div(
    [
        html.H1(
            "Michaelis-Menten (uncompetitive inhibition) by concentration of inhibitor"
        ),
        html.Label("Select concentration of inhibitor:"),
        dcc.Dropdown(
            id="i-dropdown",
            options=[{"label": val, "value": val} for val in u_obs["i"].unique()],
            value=u_obs["i"].unique()[0],
        ),
        dcc.Graph(id="scatter-plot"),
    ]
)


# Callback to update the plot
@app3.callback(Output("scatter-plot", "figure"), [Input("i-dropdown", "value")])
def update_plot(selected_i):
    filtered_data = u_obs[u_obs["i"] == selected_i]
    fig = px.scatter(
        filtered_data,
        x=np.log(filtered_data["s"]),
        y="ui_v",
        color="s",
        title=f"Scatter plot for inhibitor concentration = {selected_i}",
        labels={
            "ui_v": "Reaction velocity (v)",
            "x": "Log of substrate concentration",
            "s": "Substrate concentration (s)",
        },
        trendline="ols",
        trendline_options=dict(log_x=True),
    )
    return fig


# Run the app
if __name__ == "__main__":
    app3.run_server(debug=True)